# création du dataset #

In [79]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sympy.codegen import Print
import requests
# === Paramètres ===
file_path = "C:/Users\hugom\OneDrive\Documents\Stage_2025\data\Liste de journal de tendance multiple 22072025.xlsx"

# === Chargement des données ===

df = pd.read_excel(file_path)
file_conso_path = "C:/Users\hugom\OneDrive\Documents\Stage_2025\data\THHeating.xlsx"
cg = pd.read_excel(file_conso_path)

file_conso_path = "C:/Users\hugom\OneDrive\Documents\Stage_2025\data\TH.xlsx"
cw = pd.read_excel(file_conso_path)
print(cg)
print(cw)

          FIFTEEN_MINUTE  gas_consumption.actual  gaztot.actual
0    2025-07-07 00:00:00                    0.00        105.432
1    2025-07-07 00:15:00                    0.00        112.308
2    2025-07-07 00:30:00                    0.00        110.016
3    2025-07-07 00:45:00                    0.00        107.724
4    2025-07-07 01:00:00                    0.00        105.432
...                  ...                     ...            ...
1435 2025-07-21 22:45:00                   10.36        123.814
1436 2025-07-21 23:00:00                    0.00        106.578
1437 2025-07-21 23:15:00                   82.88        195.188
1438 2025-07-21 23:30:00                    0.00        106.578
1439 2025-07-21 23:45:00                    0.00        112.308

[1440 rows x 3 columns]
          FIFTEEN_MINUTE  eau_final.actual
0    2025-07-07 00:00:00               2.8
1    2025-07-07 00:15:00               0.1
2    2025-07-07 00:30:00               0.2
3    2025-07-07 00:45:00           

C:\Users\hugom\OneDrive\Documents\Stage_2025\Stage_env\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\hugom\OneDrive\Documents\Stage_2025\Stage_env\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [80]:
data = []
k= 0
for i in df:
    data.append(df[i])
cmd_indx = [13,14,15,16,17,18,19,20,54,58,62,66] #liste des commandes utiles
cmd = []
for i in cmd_indx:
    cmd.append(data[i])
print(np.shape(cmd))
#cmd = matrices des commandes en fonction du temps

(12, 6000)


In [81]:
k = 0
for i in df:
    k+=1
    if k in cmd_indx:
        print(i)

Depart_Chaudiere Interval Trend Log
Distribution__Ht_+2_Pmp2A_Cmd
Distribution__Ht_+2_Pmp2B_Cmd
Distribution__Ht_+27_Pmp1A_Cmd
Distribution__Ht_+27_Pmp1B_Cmd
Distribution__Ht_-4_Pmp3A_Cmd
Distribution__Ht_-4_Pmp3B_Cmd
Distribution__Ht_Rad_Pmp1_Cmd
Production__Boiler3_Power_Fb
Production_Boiler1_Spt_Mod
Production_Boiler2_Spt_Mod
Production_Boiler3_Spt_Mod


## création import API pour get température

In [82]:
import requests
import pandas as pd
from datetime import datetime
date = "01-01-24 09:00:00"
def get_temp(date_datetime,lat =50.8503, lon = 4.3517):
    date = str(date_datetime)
    date = date.split(" ")
    date = date[0]
    url = (
        f"https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={date}&end_date={date}"
        f"&hourly=temperature_2m"
    )

    response = requests.get(url)
    data = response.json()
    temps = data['hourly']['temperature_2m']
    timestamps = data['hourly']['time']
    df = pd.DataFrame({'time': timestamps, 'temperature': temps})
    df['time'] = pd.to_datetime(df['time'])

    return df['temperature']

date2 = datetime.strptime(date, "%d-%m-%y %H:%M:%S")
print(get_temp(date2))

0     7.9
1     6.3
2     7.1
3     7.2
4     6.8
5     6.8
6     6.6
7     6.3
8     6.0
9     6.3
10    7.5
11    8.2
12    9.0
13    7.9
14    7.9
15    7.6
16    7.1
17    7.1
18    6.9
19    7.0
20    7.1
21    7.4
22    7.7
23    7.8
Name: temperature, dtype: float64


## création classe segment d'input ##

In [83]:
class segment():
    def __init__(self,Cmd,temp_ext,time,water_cons,conso_list,pw):
        self.cmd = Cmd
        self.conso_list = conso_list.tolist()
        self.temp_ext = temp_ext
        self.time = time.tolist()
        self.water_cons = water_cons.tolist()
        self.pw = pw
        #self.occupation = occupation
    def gettime(self):

        return self.time
    def getdata(self):

        t = self.time[pw].hour
        conso = self.conso_list[pw]
        data = self.cmd,conso,self.temp_ext,self.water_cons,t
        return data
    def flatten(self):
        t = self.time[pw].hour
        data = []
        for i in self.cmd:
            for j in i:
                data.append(j)
        conso = self.conso_list[pw]
        data.append(conso)
        for i in self.temp_ext:
            data.append(i)
        data.append(self.water_cons)
        data.append(t)
        for i in self.conso_list:
            data.append(i)
        return data

### égalisation échelles de temps ###

In [84]:
from datetime import datetime
from datetime import timedelta
from IPython.display import clear_output
import time


normalized_cmd=[]
drop = []
for i in range(0,len(cg['FIFTEEN_MINUTE']),1):
    # Deux dates au format chaîne de caractères
    min = timedelta(days=5, hours=0, minutes=0, seconds=0)
    for j in range(len(data[0])):
        date1 = cg['FIFTEEN_MINUTE'][i]
        data_temp = data[0][j].replace("/","-")
        data_temp = data_temp[:6]+data_temp[8:]
        date2 = datetime.strptime(data_temp, "%d-%m-%y %H:%M:%S")
        n_min = abs(date1- date2)
        if n_min<min:
            min = n_min
            indx = j
    clear_output()
    print(cg['FIFTEEN_MINUTE'][i])
    data_temp = data[0][indx].replace("/","-")
    data_temp = data_temp[:6]+data_temp[8:]
    print(datetime.strptime(data_temp, "%d-%m-%y %H:%M:%S"))
    print(abs(min))
    if min<timedelta(days=0, hours=0, minutes=5, seconds=0):
        normalized_cmd.append(np.transpose(cmd)[indx])
    else:
        drop.append(i)
        print("🫠aucune correspondance trouvée")

    print(f"😑wala j'y travaille : {float(np.shape(normalized_cmd)[0]) / float(np.shape(cg['FIFTEEN_MINUTE'])[0]-len(drop))}")
for i in drop:
    cw = cw.drop(i).reset_index(drop=True)
    cg = cg.drop(i).reset_index(drop=True)

2025-07-21 23:45:00
2025-07-21 23:45:00
0 days 00:00:00
😑wala j'y travaille : 1.0


In [85]:
#verification des tailles
print(len(cg['FIFTEEN_MINUTE']))
print(len(cw['FIFTEEN_MINUTE']))
print(normalized_cmd[0])
print(np.transpose(cmd)[0])
#c'est inversé me demande pas pq ducoup on regle ça de la manière la plus résonnable et honnete 🫣:
normalized_cmd =  normalized_cmd[::-1]
print(normalized_cmd[0])
print(np.transpose(cmd)[0])

1400
1400
[1 0 1 0 1 0 0 0 1 1 1 1]
[1 0 0 1 1 0 0 0 1 1 1 1]
[1 0 0 1 1 0 0 0 1 1 1 1]
[1 0 0 1 1 0 0 0 1 1 1 1]


### création des segments ###

In [86]:
temp_today = {}
n_data = []
pw = 19
fw = 1
for i in range(pw,len(cg['FIFTEEN_MINUTE'])-fw,1):
    Cmd = normalized_cmd[i-pw:i+fw]
    conso_w = np.mean(cw['eau_final.actual'][i-pw:i+fw])
    conso_g = cg['gaztot.actual'][i-pw:i+fw]
    time_w = cg['FIFTEEN_MINUTE'][i-pw:i+fw]
    temp_ext = []
    for k in time_w:
        #c'est lent mais normal c'est fait exprès pcq si on va trop vite on risque de se faire ban de l'API de météo va te faire un kfé frérot ça va prendre +/- 10 min
        #update : j'ai opti le truc en fait t'as pas le temps pour un kfé

        if k.date() not in temp_today:
            time.sleep(0.02)
            temp = get_temp(k)
            temp_today[k.date()]=temp

        temp_ext.append(temp_today[k.date()][k.hour])
    seg = segment(Cmd,temp_ext,time_w,conso_w,conso_g,pw)
    n_data.append(seg)
    clear_output()
    print(f"😑wala j'y travaille : {float(len(n_data)) /float(len(cg['FIFTEEN_MINUTE'])-fw-pw)}")

😑wala j'y travaille : 1.0


In [87]:
#vérification de la taille
print(len(cg['FIFTEEN_MINUTE'])-fw-pw)
print(len(n_data))
print(n_data[0].getdata())

1380
1380
([array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1])], 105.43200000013341, [np.float64(15.1), np.float64(15.1), np.float64(15.1), np.float64(1

# processing de l'output #

## fonction de recherche de segments similaires ##

In [88]:
dataset=n_data.copy()
def search_sim(input,dataset,threshold):
    sim_ind = []
    list_comp=[1,2,3,4]#élément discriminants pour la similitude de plusieurs situations
    for i in range(len(dataset)):
        a = []
        b = []
        for j in list_comp:
            if type(input.getdata()[j]) == list:
                for k in range(len(input.getdata()[j])):
                    a.append(dataset[i].getdata()[j][k])
                    b.append(input.getdata()[j][k])
            else:
                a.append(dataset[i].getdata()[j])
                b.append(input.getdata()[j])
        a = np.array(a)
        b = np.array(b)
        if np.linalg.norm(a-b)<threshold:
            sim_ind.append(i)
    return sim_ind

print(search_sim(dataset[0],dataset,15))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, 106, 108, 111, 113, 114, 117, 118, 119, 120, 121, 122, 123, 125, 126, 127, 128, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 195, 202, 204, 205, 260, 261, 262, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 293, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 475, 476, 477, 478, 479, 480, 481, 482, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 650, 651, 652, 653, 654, 655, 656, 657, 658

## fonction d'enregistrement sous .pt ##

In [89]:
import torch
def savefile(input,output,pw):
    path = input.gettime()[pw].strftime('%y-%m-%d %H:%M:%S')
    input = input.flatten()
    input = np.array(input)
    if isinstance(input, np.ndarray):
        input_arr = np.array(input, dtype=np.float32)
        input = torch.from_numpy(input_arr)
    if not torch.isnan(input).any():

        output_arr = np.array(output, dtype=np.float32)
        output = torch.from_numpy(output_arr)
        if not torch.isnan(output).any():
            print(output)
            path = path.replace(' ', '')
            path = path.replace('-', '_')
            path = path.replace(':', '_')

            data = {
            "input": input,
            "output": output
            }
            root = "C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/"
            path = root+path

            # Sauvegarde dans un fichier .pt
            torch.save(data, f"{path}.pt")

            print(f"Fichier '{path}.pt' enregistré avec succès.")

savefile(dataset[0],[0, 1, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0],pw)

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Fichier 'C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/25_07_0705_00_00.pt' enregistré avec succès.


## création des segments pr tout le dataset ##

In [90]:
print(len(dataset))
for i in range(len(dataset)):
    sim_ind = search_sim(dataset[i],dataset,15)
    for j in sim_ind:
        min = dataset[i].getdata()[1]
        if dataset[j].getdata()[1]<min:
            min = dataset[j].getdata()[1]
            indx = j


    a = dataset[i].getdata()[0][pw]
    b = dataset[indx].getdata()[0][pw]
    chng = np.array(a)-np.array(b)
    input = dataset[i]
    for i in range(len(chng)):
        if a[i]==0:
            chng[i]=0
    savefile(input,abs(chng),pw)


1380
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
Fichier 'C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/25_07_0705_00_00.pt' enregistré avec succès.
tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])
Fichier 'C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/25_07_0705_15_00.pt' enregistré avec succès.
tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])
Fichier 'C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/25_07_0705_30_00.pt' enregistré avec succès.
tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])
Fichier 'C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/25_07_0705_45_00.pt' enregistré avec succès.
tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])
Fichier 'C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/25_07_0706_00_00.pt' enregistré avec succès.
tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])
Fichier 'C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/25_07_07

## appel d'un segment pour tester ##

In [2]:
import torch
test = torch.load('C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/25_07_2016_45_00.pt',weights_only=False)
print(test)
print(len(test['input']))
print(len(test['output']))
print(test['input'][263:283])

{'input': tensor([  1.0000,   0.0000,   1.0000,   0.0000,   1.0000,   0.0000,   0.0000,
          0.0000,   1.0000,   1.0000,   1.0000,   1.0000,   1.0000,   0.0000,
          1.0000,   0.0000,   1.0000,   0.0000,   0.0000,   0.0000,   1.0000,
          1.0000,   1.0000,   1.0000,   1.0000,   0.0000,   1.0000,   0.0000,
          1.0000,   0.0000,   0.0000,   0.0000,   1.0000,   1.0000,   1.0000,
          1.0000,   1.0000,   0.0000,   1.0000,   0.0000,   1.0000,   0.0000,
          0.0000,   0.0000,   1.0000,   1.0000,   1.0000,   1.0000,   1.0000,
          0.0000,   1.0000,   0.0000,   1.0000,   0.0000,   0.0000,   0.0000,
          1.0000,   1.0000,   1.0000,   1.0000,   1.0000,   0.0000,   1.0000,
          0.0000,   1.0000,   0.0000,   0.0000,   0.0000,   1.0000,   1.0000,
          1.0000,   1.0000,   1.0000,   0.0000,   1.0000,   0.0000,   1.0000,
          0.0000,   0.0000,   0.0000,   1.0000,   1.0000,   1.0000,   1.0000,
          1.0000,   0.0000,   1.0000,   0.0000,   1.00

[2, 3]
